# 🚂 02 — Training

**Purpose:** Train models on full images or hybrid ROI crops.

**Sections:**
1. Inline Setup
2. Copy Data to /content (full images OR hybrid crops)
3. Register Models
4. Training Configuration & Execution
5. Learning Curves Visualization

**Prerequisites:** Manifests and splits exist on Drive (from 01_data_preparation.ipynb or prior run)


## 🔧 Section 1: Inline Setup


In [ ]:
# --- INLINE SETUP ---
import os, subprocess, sys

# Config
REPO_URL       = "https://github.com/ClaudiaCPach/CNNs-distracted-driving"
REPO_DIRNAME   = "CNNs-distracted-driving"
BRANCH         = "main"
PROJECT_ROOT   = f"/content/{REPO_DIRNAME}"
DRIVE_PATH     = "/content/drive/MyDrive/TFM"
DRIVE_DATA_ROOT = f"{DRIVE_PATH}/data"
FAST_DATA      = "/content/data"
DATASET_ROOT   = DRIVE_DATA_ROOT
OUT_ROOT       = f"{DRIVE_PATH}/outputs"
CKPT_ROOT      = f"{DRIVE_PATH}/checkpoints"

# Mount Drive
from google.colab import drive
drive.mount('/content/drive', force_remount=False)

# Clone/update repo
def sh(cmd):
    print(f"$ {cmd}")
    rc = subprocess.call(cmd, shell=True, executable="/bin/bash")
    if rc != 0:
        raise RuntimeError(f"Command failed: {cmd}")

if os.path.isdir(PROJECT_ROOT):
    sh(f"cd {PROJECT_ROOT} && git pull --rebase origin {BRANCH}")
else:
    sh(f"git clone --branch {BRANCH} {REPO_URL} {PROJECT_ROOT}")

# Install
sh(f"pip install -q -e {PROJECT_ROOT}")
!pip -q install timm

# Set env vars
os.environ["DRIVE_PATH"] = DRIVE_PATH
os.environ["DATASET_ROOT"] = DATASET_ROOT
os.environ["OUT_ROOT"] = OUT_ROOT
os.environ["CKPT_ROOT"] = CKPT_ROOT
os.environ["FAST_DATA"] = FAST_DATA

sys.path.insert(0, PROJECT_ROOT)
sys.path.insert(0, os.path.join(PROJECT_ROOT, "src"))

# GPU check
!nvidia-smi || echo "No GPU"
print("✅ Inline setup complete")


## ⚡ Section 2: Fast Data Loading from Tar Archives

Choose ONE of the options below based on what you're training on.

**⚡ Speed Boost:** These cells use tar archives for fast loading (~5 min instead of ~2 hours).
Tar archives must be created first using 01_data_preparation.ipynb (Section 4).


In [ ]:
# ⚡ OPTION A: Extract HYBRID CROPS from tar archive (for face/face_hands training)
# Uses tar archive for fast loading (~5 min vs 2+ hours)

import os
import shutil
from pathlib import Path
import importlib
from ddriver.data.fastcopy import fast_copy_from_tar

HYBRID_VARIANT = "face"  # face | face_hands

LOCAL_ROOT = Path("/content/data/hybrid")
DRIVE_ROOT = Path(OUT_ROOT) / "hybrid"
LOCAL_VARIANT_DIR = LOCAL_ROOT / HYBRID_VARIANT

# Tar archive path (created in 01_data_preparation.ipynb)
TAR_PATH = DRIVE_ROOT / f"hybrid_{HYBRID_VARIANT}.tar"

# Check if already extracted
if LOCAL_VARIANT_DIR.exists() and any(LOCAL_VARIANT_DIR.rglob("*.jpg")):
    jpg_count = sum(1 for _ in LOCAL_VARIANT_DIR.rglob("*.jpg"))
    print(f"✅ Hybrid crops already in /content ({jpg_count} jpgs). Skipping.")
elif not TAR_PATH.exists():
    raise FileNotFoundError(
        f"Tar archive not found: {TAR_PATH}\n"
        "Run 01_data_preparation.ipynb Section 4 to create tar archives first."
    )
else:
    print(f"📦 Extracting {HYBRID_VARIANT} crops from tar archive...")
    result = fast_copy_from_tar(
        tar_path_on_drive=TAR_PATH,
        dest_dir=LOCAL_VARIANT_DIR,
        remove_tar_after=True,
        verbose=True,
    )
    print(f"   ✅ Extracted {result.get('n_files', '?')} files")

    # Copy CSVs (these are small, direct copy is fine)
    for fname in [f"manifest_{HYBRID_VARIANT}.csv", f"train_{HYBRID_VARIANT}.csv",
                  f"val_{HYBRID_VARIANT}.csv", f"test_{HYBRID_VARIANT}.csv"]:
        src = DRIVE_ROOT / fname
        if src.exists():
            shutil.copy2(src, LOCAL_ROOT / fname)
            print(f"   Copied {fname}")

# Update env vars
os.environ["HYBRID_ROOT_LOCAL"] = str(LOCAL_ROOT)
os.environ["DATASET_ROOT"] = str(LOCAL_ROOT)
from ddriver import config as _cfg
importlib.reload(_cfg)
print(f"\n✅ DATASET_ROOT = {os.environ['DATASET_ROOT']}")


In [ ]:
# ⚡ OPTION B: Extract FULL IMAGES from tar archive (for full-frame training)
# Uses pre-compressed tar archive for fast loading (~5 min vs 2+ hours)
# Skip this if using hybrid crops above

import os
import importlib
from pathlib import Path
from ddriver.data.fastcopy import fast_copy_from_tar

# Tar archive path (created in 01_data_preparation.ipynb)
TAR_PATH = Path(DRIVE_DATA_ROOT) / "full_compressed.tar"
DST_ROOT = Path(FAST_DATA) / "auc.distracted.driver.dataset_v2"

# Check if already extracted
if DST_ROOT.exists() and any(DST_ROOT.rglob("*.jpg")):
    jpg_count = sum(1 for _ in DST_ROOT.rglob("*.jpg"))
    print(f"✅ Full images already in /content ({jpg_count} jpgs). Skipping.")
elif not TAR_PATH.exists():
    raise FileNotFoundError(
        f"Tar archive not found: {TAR_PATH}\n"
        "Run 01_data_preparation.ipynb Section 4b to create the full-frame tar archive first."
    )
else:
    print("📦 Extracting full-frame images from tar archive...")
    result = fast_copy_from_tar(
        tar_path_on_drive=TAR_PATH,
        dest_dir=DST_ROOT,
        remove_tar_after=True,
        verbose=True,
    )
    print(f"   ✅ Extracted {result.get('n_files', '?')} files")

os.environ["DATASET_ROOT"] = str(FAST_DATA)
from ddriver import config as _cfg
importlib.reload(_cfg)
print(f"✅ DATASET_ROOT = {os.environ['DATASET_ROOT']}")


## 📦 Section 3: Register Models


In [ ]:
# Register models from timm
from ddriver.models import registry

registry.register_timm_backbone("efficientnet_b0")
# registry.register_timm_backbone("convnext_tiny")
# registry.register_timm_backbone("resnet50")

print("Available models:", registry.available_models()[:10])


## 🚂 Section 4: Training Configuration & Execution


In [ ]:
# 🚂 TRAINING CONFIGURATION
import os
import subprocess, textwrap, json, time, threading
from pathlib import Path

# ============== EXPERIMENT CONFIG ==============
RUN_TAG = "effb0_face_v1"           # <<<< CHANGE for each experiment
MODEL_NAME = "efficientnet_b0"
SEED = 42                           # <<<< CHANGE for stability runs (e.g., 42, 123, 456)

# Training hyperparameters
EPOCHS = 30
BATCH_SIZE = 32
NUM_WORKERS = 2
IMAGE_SIZE = 224
LR = 3e-4
LABEL_SMOOTHING = 0.05
USE_TINY_SPLIT = False

# Data source: choose ONE
USE_HYBRID = True                   # True = use hybrid crops
ROI_VARIANT = "face"                # face | face_hands (only if USE_HYBRID=True)

# Control split selection (for 5-run experimental plan)
# Set to None for natural runs, or "facesubset" / "fhsubset" / "both" for control runs
# NOTE: Control splits only work with full-frame (USE_HYBRID=False)
USE_CONTROL_SPLIT = None            # None | "facesubset" | "fhsubset" | "both"

# Validate settings
if USE_CONTROL_SPLIT and USE_HYBRID:
    raise ValueError("Control splits require full-frame training. Set USE_HYBRID=False to use control splits.")

# ============== BUILD PATHS ==============
if USE_HYBRID:
    hybrid_root = Path(os.environ.get("HYBRID_ROOT_LOCAL", Path(OUT_ROOT) / "hybrid"))
    manifest_csv = hybrid_root / f"manifest_{ROI_VARIANT}.csv"
    train_split = f"train_{ROI_VARIANT}.csv" if not USE_TINY_SPLIT else f"train_small_{ROI_VARIANT}.csv"
    train_csv = hybrid_root / train_split
    val_csv = hybrid_root / f"val_{ROI_VARIANT}.csv"
    test_csv = hybrid_root / f"test_{ROI_VARIANT}.csv"
    print(f"🔀 Using Hybrid crops: {ROI_VARIANT}")
    print(f"   hybrid_root = {hybrid_root}")
else:
    manifest_csv = Path(OUT_ROOT) / "manifests" / "manifest.csv"
    # Handle control splits for 5-run experimental plan
    if USE_CONTROL_SPLIT:
        control_root = Path(OUT_ROOT) / "splits" / "control"
        train_csv = control_root / f"train_{USE_CONTROL_SPLIT}.csv"
        val_csv = control_root / f"val_{USE_CONTROL_SPLIT}.csv"
        test_csv = control_root / f"test_{USE_CONTROL_SPLIT}.csv"
        print(f"📷 Using full-frame images with CONTROL SPLIT: {USE_CONTROL_SPLIT}")
        print(f"   (Filtered to {USE_CONTROL_SPLIT} IDs for fair comparison)")
    else:
        train_split = "train_small.csv" if USE_TINY_SPLIT else "train.csv"
        train_csv = Path(OUT_ROOT) / "splits" / train_split
        val_csv = Path(OUT_ROOT) / "splits" / "val.csv"
        test_csv = Path(OUT_ROOT) / "splits" / "test.csv"
        print("📷 Using full-frame images")

# Update DATASET_ROOT for hybrid
if USE_HYBRID:
    _roi_root = hybrid_root
    if str(_roi_root).startswith("/content/data"):
        import importlib
        os.environ["DATASET_ROOT"] = str(_roi_root)
        from ddriver import config as _cfg
        importlib.reload(_cfg)
        print(f"   ⚡ DATASET_ROOT = {os.environ['DATASET_ROOT']}")

print(f"\n📋 Training config:")
print(f"   RUN_TAG: {RUN_TAG}")
print(f"   Model: {MODEL_NAME}")
print(f"   Seed: {SEED}")
print(f"   Epochs: {EPOCHS}, Batch: {BATCH_SIZE}, LR: {LR}")
if USE_CONTROL_SPLIT:
    print(f"   Control Split: {USE_CONTROL_SPLIT}")


In [ ]:
# 🚂 RUN TRAINING



# ============== FIX: Convert absolute Drive paths to relative ==============
# This ensures DATASET_ROOT is respected even if manifest has absolute paths
import pandas as pd

def _make_relative(path_str: str) -> str:
    """Convert absolute path to relative from dataset marker."""
    marker = "auc.distracted.driver.dataset_v2"
    if marker in path_str:
        idx = path_str.find(marker)
        return path_str[idx:]  # Keep from mabrker onwards
    return path_str

# Patch the split CSVs to use relative paths
for csv_path in [train_csv, val_csv, test_csv]:
    if Path(csv_path).exists():
        df = pd.read_csv(csv_path)
        if "path" in df.columns:
            sample_path = df["path"].iloc[0] if len(df) > 0 else ""
            if "/content/drive" in sample_path:
                print(f"⚠️  Converting {csv_path.name} paths to relative...")
                df["path"] = df["path"].apply(_make_relative)
                # Write to local temp location
                local_csv = Path("/content") / csv_path.name
                df.to_csv(local_csv, index=False)
                if "train" in str(csv_path):
                    train_csv = local_csv
                elif "val" in str(csv_path):
                    val_csv = local_csv
                elif "test" in str(csv_path):
                    test_csv = local_csv
                print(f"   ✅ Wrote {local_csv} with relative paths")

# Also verify DATASET_ROOT
print(f"\n🔍 DATASET_ROOT = {os.environ.get('DATASET_ROOT')}")
fast_images = Path("/content/data/auc.distracted.driver.dataset_v2")
if fast_images.exists():
    print(f"   ✅ Fast storage contains images")
else:
    print(f"   ❌ WARNING: {fast_images} does not exist!")

train_cmd = textwrap.dedent(f"""
cd {PROJECT_ROOT}
python -m src.ddriver.cli.train \
    --model-name {MODEL_NAME} \
    --epochs {EPOCHS} \
    --batch-size {BATCH_SIZE} \
    --num-workers {NUM_WORKERS} \
    --image-size {IMAGE_SIZE} \
    --lr {LR} \
    --weight-decay .01 \
    --optimizer adamw \
    --label-smoothing {LABEL_SMOOTHING} \
    --seed {SEED} \
    --out-tag {RUN_TAG} \
    --manifest-csv {manifest_csv} \
    --train-csv {train_csv} \
    --val-csv {val_csv} \
    --test-csv {test_csv}
""")

print("Running training command:\n", train_cmd)

proc = subprocess.Popen(train_cmd, shell=True, text=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT)

# GPU monitor thread
def _gpu_monitor():
    while proc.poll() is None:
        try:
            stats = subprocess.check_output(
                "nvidia-smi --query-gpu=utilization.gpu,memory.used,memory.total --format=csv,nounits,noheader",
                shell=True,
            ).decode("utf-8").strip()
            print(f"[GPU] {stats}")
        except:
            pass
        time.sleep(5)

monitor = threading.Thread(target=_gpu_monitor, daemon=True)
monitor.start()

for line in proc.stdout:
    print(line, end="")

proc.wait()
print("\n✅ Training complete!")


## 📈 Section 5: Learning Curves Visualization


In [ ]:
# 📈 Display training metrics and learning curves
import json
import matplotlib.pyplot as plt
from pathlib import Path

run_base = Path(CKPT_ROOT) / "runs" / RUN_TAG
all_runs = sorted(run_base.glob("*/"))
if not all_runs:
    raise FileNotFoundError(f"No run folders found under {run_base}")
latest_run = all_runs[-1]

history_path = latest_run / "history.json"
if not history_path.exists():
    raise FileNotFoundError(f"history.json not found in {latest_run}")

history = json.loads(history_path.read_text()).get("history", [])

print(f"📊 Epoch metrics for run: {latest_run.name}")
for record in history:
    train_metrics = record.get("train", {})
    val_metrics = record.get("val", {}) or {}
    train_loss = train_metrics.get("loss")
    train_acc = train_metrics.get("accuracy")
    val_loss = val_metrics.get("loss")
    val_acc = val_metrics.get("accuracy")
    val_str = f"val_loss={val_loss:.4f} acc={val_acc:.4f}" if val_loss else "val_loss=— val_acc=—"
    print(f"  Epoch {record['epoch']:>2}: train_loss={train_loss:.4f} acc={train_acc:.4f}  {val_str}")

# Plot learning curves
epochs = [r["epoch"] for r in history]
train_loss = [r["train"]["loss"] for r in history]
train_acc = [r["train"]["accuracy"] for r in history]
val_loss = [(r.get("val") or {}).get("loss") for r in history]
val_acc = [(r.get("val") or {}).get("accuracy") for r in history]

val_epochs = [e for e, v in zip(epochs, val_loss) if v is not None]
val_loss_f = [v for v in val_loss if v is not None]
val_acc_f = [v for v in val_acc if v is not None]

fig, axes = plt.subplots(1, 2, figsize=(12, 4), dpi=140)

axes[0].plot(epochs, train_acc, label="Training Accuracy")
if val_acc_f:
    axes[0].plot(val_epochs, val_acc_f, label="Validation Accuracy")
axes[0].set_title("Accuracy Curves")
axes[0].set_xlabel("Epochs")
axes[0].set_ylabel("Accuracy")
axes[0].legend()

axes[1].plot(epochs, train_loss, label="Training Loss")
if val_loss_f:
    axes[1].plot(val_epochs, val_loss_f, label="Validation Loss")
axes[1].set_title("Loss Curves")
axes[1].set_xlabel("Epochs")
axes[1].set_ylabel("Loss")
axes[1].legend()

plt.tight_layout()
out_png = latest_run / "learning_curves.png"
fig.savefig(out_png, bbox_inches="tight")
plt.show()
print("✅ Saved:", out_png)


In [ ]:
# 📊 Annex Figures: Training Curves Comparison
# Generates Figure A.1 (Face vs Full-frame FaceSubset) and Figure A.2 (Face+Hands vs Full-frame FHSubset)

import json
import matplotlib.pyplot as plt
from pathlib import Path
import numpy as np

# ============== CONFIGURATION ==============
CKPT_ROOT = Path("/content/drive/MyDrive/TFM/checkpoints")

# Run tags for each experiment
RUN_TAGS = {
    "face": "effb0_face_1_11",
    "face_hands": "effb0_face_hands_1_11",
    "full_facesubset": "effb0_full_image_facesubset_1_10",
    "full_fhsubset": "effb0_full_image_fhsubset_1_10",
}

# Color scheme (consistent across all figures)
COLORS = {
    "face": "tab:blue",
    "full_facesubset": "tab:orange",
    "face_hands": "tab:green",
    "full_fhsubset": "tab:red",
}

# ============== HELPER FUNCTIONS ==============

def load_history(run_tag: str) -> dict:
    """Load history.json for a given run tag, returning the latest run."""
    run_base = CKPT_ROOT / "runs" / run_tag
    all_runs = sorted(run_base.glob("*/"))
    if not all_runs:
        raise FileNotFoundError(f"No run folders found under {run_base}")
    latest_run = all_runs[-1]

    history_path = latest_run / "history.json"
    if not history_path.exists():
        raise FileNotFoundError(f"history.json not found in {latest_run}")

    data = json.loads(history_path.read_text())
    history = data.get("history", [])

    # Extract arrays
    epochs = [r["epoch"] for r in history]
    train_acc = [r["train"]["accuracy"] for r in history]
    train_loss = [r["train"]["loss"] for r in history]
    val_acc = [(r.get("val") or {}).get("accuracy") for r in history]
    val_loss = [(r.get("val") or {}).get("loss") for r in history]

    # Filter out None values for validation (in case some epochs skipped validation)
    val_epochs = [e for e, v in zip(epochs, val_acc) if v is not None]
    val_acc_clean = [v for v in val_acc if v is not None]
    val_loss_clean = [v for v in val_loss if v is not None]

    return {
        "run_dir": latest_run,
        "epochs": epochs,
        "train_acc": train_acc,
        "train_loss": train_loss,
        "val_epochs": val_epochs,
        "val_acc": val_acc_clean,
        "val_loss": val_loss_clean,
    }


def compute_axis_limits(histories: list, metric: str, padding: float = 0.05):
    """Compute shared y-axis limits across multiple histories for a given metric."""
    all_values = []
    for h in histories:
        if metric == "acc":
            all_values.extend(h["train_acc"])
            all_values.extend(h["val_acc"])
        else:  # loss
            all_values.extend(h["train_loss"])
            all_values.extend(h["val_loss"])

    ymin, ymax = min(all_values), max(all_values)
    margin = (ymax - ymin) * padding
    return (ymin - margin, ymax + margin)


def create_comparison_figure(
    hist_roi: dict,
    hist_full: dict,
    roi_key: str,
    full_key: str,
    roi_label: str,
    full_label: str,
    title_suffix: str,
    acc_ylim: tuple,
    loss_ylim: tuple,
    output_path: Path,
):
    """Create a 1x2 figure comparing ROI vs Full-frame training curves."""

    fig, axes = plt.subplots(1, 2, figsize=(12, 4.5), sharex=True, dpi=150)

    # --- Left subplot: Accuracy ---
    ax = axes[0]

    # ROI model
    ax.plot(hist_roi["epochs"], hist_roi["train_acc"],
            color=COLORS[roi_key], linestyle='-', linewidth=1.5,
            label=f"{roi_label} – train")
    ax.plot(hist_roi["val_epochs"], hist_roi["val_acc"],
            color=COLORS[roi_key], linestyle='--', linewidth=1.5,
            label=f"{roi_label} – val")

    # Full-frame model
    ax.plot(hist_full["epochs"], hist_full["train_acc"],
            color=COLORS[full_key], linestyle='-', linewidth=1.5,
            label=f"{full_label} – train")
    ax.plot(hist_full["val_epochs"], hist_full["val_acc"],
            color=COLORS[full_key], linestyle='--', linewidth=1.5,
            label=f"{full_label} – val")

    ax.set_title(f"{title_suffix} – Accuracy", fontsize=11, fontweight='bold')
    ax.set_xlabel("Epoch")
    ax.set_ylabel("Accuracy")
    ax.set_ylim(acc_ylim)
    ax.legend(loc='lower right', fontsize=8)
    ax.grid(True, alpha=0.3)

    # --- Right subplot: Loss ---
    ax = axes[1]

    # ROI model
    ax.plot(hist_roi["epochs"], hist_roi["train_loss"],
            color=COLORS[roi_key], linestyle='-', linewidth=1.5,
            label=f"{roi_label} – train")
    ax.plot(hist_roi["val_epochs"], hist_roi["val_loss"],
            color=COLORS[roi_key], linestyle='--', linewidth=1.5,
            label=f"{roi_label} – val")

    # Full-frame model
    ax.plot(hist_full["epochs"], hist_full["train_loss"],
            color=COLORS[full_key], linestyle='-', linewidth=1.5,
            label=f"{full_label} – train")
    ax.plot(hist_full["val_epochs"], hist_full["val_loss"],
            color=COLORS[full_key], linestyle='--', linewidth=1.5,
            label=f"{full_label} – val")

    ax.set_title(f"{title_suffix} – Loss", fontsize=11, fontweight='bold')
    ax.set_xlabel("Epoch")
    ax.set_ylabel("Cross-Entropy Loss")
    ax.set_ylim(loss_ylim)
    ax.legend(loc='upper right', fontsize=8)
    ax.grid(True, alpha=0.3)

    plt.tight_layout()
    fig.savefig(output_path, bbox_inches='tight', dpi=150)
    plt.show()
    print(f"✅ Saved: {output_path}")
    return fig


# ============== MAIN EXECUTION ==============

print("📂 Loading training histories...")

# Load all histories
hist_face = load_history(RUN_TAGS["face"])
hist_face_hands = load_history(RUN_TAGS["face_hands"])
hist_full_facesubset = load_history(RUN_TAGS["full_facesubset"])
hist_full_fhsubset = load_history(RUN_TAGS["full_fhsubset"])

print(f"   ✓ Face ROI: {len(hist_face['epochs'])} epochs")
print(f"   ✓ Face+Hands ROI: {len(hist_face_hands['epochs'])} epochs")
print(f"   ✓ Full-frame (Face IDs): {len(hist_full_facesubset['epochs'])} epochs")
print(f"   ✓ Full-frame (FH IDs): {len(hist_full_fhsubset['epochs'])} epochs")

# Compute shared axis limits across ALL runs for fair visual comparison
all_histories = [hist_face, hist_face_hands, hist_full_facesubset, hist_full_fhsubset]
acc_ylim = compute_axis_limits(all_histories, "acc", padding=0.03)
loss_ylim = compute_axis_limits(all_histories, "loss", padding=0.05)

print(f"\n📐 Shared axis limits:")
print(f"   Accuracy: {acc_ylim[0]:.3f} – {acc_ylim[1]:.3f}")
print(f"   Loss: {loss_ylim[0]:.3f} – {loss_ylim[1]:.3f}")

# ============== FIGURE A.1: Face vs Full-frame (Face IDs) ==============
print("\n" + "="*60)
print("📊 Creating Annex Figure A.1: Face vs Full-frame (Face IDs)")
print("="*60)

output_a1 = hist_face["run_dir"] / "annex_A1_face_vs_full_faceIDs_curves.png"

create_comparison_figure(
    hist_roi=hist_face,
    hist_full=hist_full_facesubset,
    roi_key="face",
    full_key="full_facesubset",
    roi_label="Face ROI",
    full_label="Full-frame (Face IDs)",
    title_suffix="Face vs Full-frame (Face IDs)",
    acc_ylim=acc_ylim,
    loss_ylim=loss_ylim,
    output_path=output_a1,
)

# ============== FIGURE A.2: Face+Hands vs Full-frame (FH IDs) ==============
print("\n" + "="*60)
print("📊 Creating Annex Figure A.2: Face+Hands vs Full-frame (FH IDs)")
print("="*60)

output_a2 = hist_face_hands["run_dir"] / "annex_A2_fh_vs_full_fhIDs_curves.png"

create_comparison_figure(
    hist_roi=hist_face_hands,
    hist_full=hist_full_fhsubset,
    roi_key="face_hands",
    full_key="full_fhsubset",
    roi_label="Face+Hands ROI",
    full_label="Full-frame (FH IDs)",
    title_suffix="Face+Hands vs Full-frame (FH IDs)",
    acc_ylim=acc_ylim,
    loss_ylim=loss_ylim,
    output_path=output_a2,
)

print("\n" + "="*60)
print("✅ All Annex figures generated!")
print(f"   A.1: {output_a1}")
print(f"   A.2: {output_a2}")
print("="*60)

## ✅ Training Complete!

**Outputs saved to Drive:**
- `CKPT_ROOT/runs/{RUN_TAG}/{timestamp}/best.pt` — Best checkpoint
- `CKPT_ROOT/runs/{RUN_TAG}/{timestamp}/history.json` — Training history
- `CKPT_ROOT/runs/{RUN_TAG}/{timestamp}/learning_curves.png` — Curves figure

**Next steps:**
- Run **03_evaluation.ipynb** to generate predictions and metrics


In [ ]:
# 📝 Training summary → Google Sheet
!pip -q install gspread

import json
from pathlib import Path

import gspread
from google.colab import auth
import google.auth

auth.authenticate_user()
creds, _ = google.auth.default()
gc = gspread.authorize(creds)

TRAIN_SHEET_NAME = "TFM Train Logs"   # create this sheet/tab ahead of time
TRAIN_WORKSHEET = "Sheet1"

run_base = Path(CKPT_ROOT) / "runs" / RUN_TAG
all_runs = sorted(run_base.glob("*/"))
if not all_runs:
    raise FileNotFoundError(f"No run folders found under {run_base}")
latest_run = all_runs[-1]
print(f"Logging training summary for run folder: {latest_run}")

history_path = latest_run / "history.json"
if not history_path.exists():
    raise FileNotFoundError(f"history.json not found under {latest_run}")

history_records = json.loads(history_path.read_text()).get("history", [])
if not history_records:
    raise ValueError(f"history.json under {latest_run} has no records.")

params_path = latest_run / "params.json"
params = json.loads(params_path.read_text()) if params_path.exists() else {}

model_name = params.get("model_name", MODEL_NAME)
epochs_cfg = params.get("epochs", EPOCHS)
batch_cfg = params.get("batch_size", BATCH_SIZE)
lr_cfg = params.get("lr", LR)
#lr_drop_epoch_cfg = params.get("lr_drop_epoch", LR_DROP_EPOCH)
#lr_drop_factor_cfg = params.get("lr_drop_factor", LR_DROP_FACTOR)
image_size_cfg = params.get("image_size", IMAGE_SIZE)
num_workers_cfg = params.get("num_workers", NUM_WORKERS)
use_tiny_cfg = params.get("use_tiny_split", USE_TINY_SPLIT)


def _best_metric(records, split: str) -> tuple[dict, float | None]:
    best_epoch = None
    best_acc = None
    for rec in records:
        split_metrics = rec.get(split) or {}
        acc = split_metrics.get("accuracy")
        if acc is None:
            continue
        if best_acc is None or acc > best_acc:
            best_acc = acc
            best_epoch = rec.get("epoch")
    final_metrics = records[-1].get(split) or {}
    final_acc = final_metrics.get("accuracy")
    return {"epoch": best_epoch, "accuracy": best_acc}, final_acc


best_train, final_train = _best_metric(history_records, "train")
best_val, final_val = _best_metric(history_records, "val")

row = [
    RUN_TAG,
    latest_run.name,
    model_name,
    epochs_cfg,
    batch_cfg,
    lr_cfg,
    #lr_drop_epoch_cfg,
    #lr_drop_factor_cfg,
    image_size_cfg,
    num_workers_cfg,
    use_tiny_cfg,
    best_train["epoch"] if best_train["epoch"] is not None else "",
    round(best_train["accuracy"], 4) if best_train["accuracy"] is not None else "",
    best_val["epoch"] if best_val["epoch"] is not None else "",
    round(best_val["accuracy"], 4) if best_val["accuracy"] is not None else "",
    round(final_train, 4) if final_train is not None else "",
    round(final_val, 4) if final_val is not None else "",
]

ws = gc.open(TRAIN_SHEET_NAME).worksheet(TRAIN_WORKSHEET)
ws.append_row(row, value_input_option="USER_ENTERED")
print(f"Appended training summary for {latest_run.name} ✅")

